# SolsysGen & WaterIO
## Project presentation notebook (tutorial + demo)

This notebook is a guided demo of:

- **SolsysGen** (Solar System Generator): deterministic heliocentric systems (**2D circular Keplerian orbits**)
- **WaterIO** (Waterfall Input and Output): checkpointing NumPy arrays to compressed **`.npz`** files


---

### Roadmap

1. Imports and setup  
2. Build a minimal system (procedural generation)  
3. Visualize orbits (figures)  
4. Time stepping + tracking (figures)  
5. Kepler formulas (equations)  
6. JSON roundtrip  
7. WaterIO checkpoint roundtrip (figures)  
8. Optional native/Cython smoke check

---

## What problem does this project solve?

When learning orbital mechanics, it is often hard to connect equations,
simulation code, and saved results in a clear and reproducible way.

This project provides:
- a simple and deterministic orbital model,
- clear visualizations of motion,
- and reliable storage of simulation results.

The focus is on understanding and reproducibility, not physical completeness.



---

## 1. Imports and setup

Run this once to import the public APIs and basic plotting tools.


In [ ]:
from __future__ import annotations

import math
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from solsysgen import SolarSystem, Sun, generate_planets
from solsysgen.constants import AU_M, DAY_S, YEAR_S
from solsysgen.io import load_json, save_json
from solsysgen.kepler import circular_speed_mps, period_s
from waterio import load_checkpoint, save_checkpoint

# Output directory for generated files (JSON + NPZ)
OUT_DIR = Path("_demo_outputs")
OUT_DIR.mkdir(exist_ok=True)

print("Setup OK. Output dir:", OUT_DIR.resolve())


---

## 2. Conceptual overview (what the project does)

### SolsysGen (simulation)
SolsysGen models a simple heliocentric system:

- circular, coplanar **2D** orbits
- explicit time stepping (`SolarSystem.step(dt)`)
- deterministic procedural generation (`generate_planets(...)`)
- clarity over realism (no N-body gravity, no perturbations)

Core objects:

- `Sun` (central body)
- `Planet` (orbital distance + phase; speed/period derived from Kepler)
- `SolarSystem` (container; stepping + serialization)

### WaterIO (checkpointing)
WaterIO provides helpers for saving/loading NumPy arrays:

- `save_checkpoint(path, **arrays)`
- `load_checkpoint(path) -> dict[str, np.ndarray]`

Useful for storing simulation outputs such as **positions over time**.


---

## 3. Minimal generated system

Create a Sun, generate planets deterministically, and inspect the result.


In [ ]:
sun = Sun()
planets = generate_planets(sun, 5, seed=1, inner_au=0.6, outer_au=12.0)
system = SolarSystem(sun=sun, planets=planets)

len(system), [p.name for p in system.planets]


### 3.1 Summary table (as Python objects)

Distances are shown in AU, periods in days/years.


In [ ]:
rows = []
for p in system.planets:
    rows.append(
        {
            "name": p.name,
            "kind": p.kind,
            "a_AU": p.distance_m / AU_M,
            "T_days": p.period_s / DAY_S,
            "T_years": p.period_s / YEAR_S,
            "v_kms": p.orbital_speed_mps / 1000.0,
        }
    )

rows


### 3.2 Derived planet statistics (sanity check)

We summarize key orbital properties to verify expected trends:
inner planets should move faster and have shorter periods,
while outer planets should move slower and have longer periods.


In [ ]:
stats = []
for p in system.planets:
    stats.append((
        p.name,
        p.kind,
        round(p.distance_m / AU_M, 2),
        round(p.period_s / YEAR_S, 2),
        round(p.orbital_speed_mps / 1000.0, 2),
    ))

stats


**What to observe:**  
As orbital distance increases, orbital period increases and orbital speed decreases,
which is consistent with Kepler’s laws.


### 3.3 Figure: orbital radii distribution

Procedural generation uses roughly log-spaced orbital distances.


In [ ]:
a_au = np.array([p.distance_m / AU_M for p in system.planets], dtype=float)

plt.figure()
plt.plot(range(1, len(a_au) + 1), a_au, marker="o")
plt.xlabel("Planet index")
plt.ylabel("Orbital distance (AU)")
plt.title("Procedurally generated orbital distances")
plt.show()

plt.figure()
plt.plot(range(1, len(a_au) + 1), np.log10(a_au), marker="o")
plt.xlabel("Planet index")
plt.ylabel("log10(Orbital distance / AU)")
plt.title("Log scale: orbital distances")
plt.show()


**What to observe:**  
Orbital distances increase with planet index. On the log plot, spacing should look roughly uniform.


### 3.4 Orbital distance vs orbital speed

We visualize how orbital speed changes with distance from the Sun.


In [ ]:
a_au = np.array([p.distance_m / AU_M for p in system.planets])
v_kms = np.array([p.orbital_speed_mps / 1000.0 for p in system.planets])

plt.figure()
plt.scatter(a_au, v_kms)
plt.xlabel("Orbital distance (AU)")
plt.ylabel("Orbital speed (km/s)")
plt.title("Orbital speed vs distance")
plt.grid(True)
plt.show()


**What to observe:**  
Planets closer to the Sun move faster, while planets farther away move slower,
which matches circular Keplerian motion.


### 3.5 Adding a custom planet (user-defined, within project limits)

We can manually add a planet with a chosen orbital radius and initial phase.
The orbit is still circular and Kepler-based, consistent with SolsysGen rules.


In [ ]:
import importlib
import solsysgen.generation as gen
importlib.reload(gen)
from solsysgen.generation import add_custom_planet

add_custom_planet(
    system,
    name="Custom Planet",
    kind="rocky",
    distance_au=1.8,
    phase_deg=45.0,
)

print("New planet count:", len(system.planets))
[p.name for p in system.planets]


In [ ]:
# Show updated summary table quickly
rows = []
for p in system.planets:
    rows.append({
        "name": p.name,
        "kind": p.kind,
        "a_AU": p.distance_m / AU_M,
        "T_years": p.period_s / YEAR_S,
        "v_kms": p.orbital_speed_mps / 1000.0,
    })
rows


In [ ]:
# Quick plot to confirm it sits at the correct radius
a_au = np.array([p.distance_m / AU_M for p in system.planets], dtype=np.float64)

plt.figure()
plt.plot(range(1, len(a_au) + 1), a_au, marker="o")
plt.xlabel("Planet index")
plt.ylabel("Orbital distance (AU)")
plt.title("Orbital distances after adding a custom planet")
plt.show()


**What to observe:**  
The custom planet appears in the table and in the distance plot at the chosen radius,
while all other planets remain unchanged.


---

## 4. Figure: orbit visualization (x–y plane)

Because orbits are circular in this project, the path should form a circle at radius `a`.
We sample multiple time steps and plot trajectories.


In [ ]:
# Fresh system to plot clean trajectories from t=0
sun_viz = Sun()
planets_viz = generate_planets(sun_viz, 5, seed=2, inner_au=0.7, outer_au=10.0)
system_viz = SolarSystem(sun=sun_viz, planets=planets_viz)

# Optional: add a custom planet for visualization
#from solsysgen.generation import add_custom_planet
#add_custom_planet(system_viz, name="Custom Planet", kind="rocky", distance_au=1.8, phase_deg=45.0)

n_steps = 300
dt = 5 * DAY_S  # 5 days per step

traj = np.zeros((n_steps + 1, len(system_viz.planets), 2), dtype=np.float64)

# Record t=0 first, then step
for t in range(n_steps + 1):
    for i, p in enumerate(system_viz.planets):
        traj[t, i, :] = p.position_m()
    if t < n_steps:
        system_viz.step(dt)

plt.figure()
for i, p in enumerate(system_viz.planets):
    plt.plot(traj[:, i, 0] / AU_M, traj[:, i, 1] / AU_M, label=p.name)

plt.scatter([0], [0], marker="*", s=120)  # Sun at origin
plt.xlabel("x (AU)")
plt.ylabel("y (AU)")
plt.title(f"Orbit trajectories (2D circular model) — steps={n_steps}, dt={dt/DAY_S:.0f} days")
plt.axis("equal")
plt.legend(loc="upper right", fontsize=8)
plt.show()


**What to observe:**  
Each planet should trace a circle. The Sun stays at the origin. Outer planets have larger circles.


---

## 5. Time stepping and orbit tracking

Pick one planet and track its position and radius across time steps.


In [ ]:
# Fresh system for tracking
sun_track = Sun()
planets_track = generate_planets(sun_track, 5, seed=1, inner_au=0.6, outer_au=12.0)
system_track = SolarSystem(sun=sun_track, planets=planets_track)

target = system_track.planets[len(system_track.planets) // 2]
target.name, target.kind, target.distance_m / AU_M


In [ ]:
steps = 120
dt = 10 * DAY_S

track_xy = np.zeros((steps + 1, 2), dtype=np.float64)
t_days = np.zeros((steps + 1,), dtype=np.float64)

t = 0.0
for i in range(steps + 1):
    x, y = target.position_m()
    track_xy[i, :] = (x, y)
    t_days[i] = t / DAY_S
    system_track.step(dt)
    t += dt

track_xy[:3], t_days[:3]


### 5.1 Figure: radius vs time (sanity check)

For a perfect circular orbit, the radius should stay constant.


In [ ]:
r = np.sqrt(track_xy[:, 0] ** 2 + track_xy[:, 1] ** 2)

plt.figure()
plt.plot(t_days, r / AU_M)
plt.xlabel("Time (days)")
plt.ylabel("Radius (AU)")
plt.title(f"Radius stability check — {target.name}")
plt.show()

float(r.min() / AU_M), float(r.max() / AU_M), float(r.mean() / AU_M)


**What to observe:**  
The radius should remain constant over time. A flat line means a stable circular orbit.


---

## 6. Kepler helpers (equations + direct evaluation)

SolsysGen uses Kepler’s third law for circular two-body orbits.

### Equations

For a planet at orbital radius \(a\) around a star of mass \(M\):

$$
T = 2\pi \sqrt{\frac{a^3}{G M}}
$$

Circular orbital speed:

$$
v = \sqrt{\frac{G M}{a}}
$$

These are implemented in:

- `period_s(a_m, M_kg)`
- `circular_speed_mps(a_m, M_kg)`


In [ ]:
a = 1.0 * AU_M
T = period_s(a, sun.mass_kg)
v = circular_speed_mps(a, sun.mass_kg)

print("For a = 1 AU:")
print("  Period (days):", T / DAY_S)
print("  Period (years):", T / YEAR_S)
print("  Speed (km/s):", v / 1000.0)


**Purpose of this check:**  
Evaluate the Kepler formulas at 1 AU to verify that the implementation produces
physically reasonable Earth-like values.

### 6.1 Extra circular-orbit formulas (angular speed and position)

For circular motion, the angular speed is:

$$
\omega = \frac{2\pi}{T}
$$

The position of a planet on a circular orbit in the x–y plane can be written as:

$$
x(t) = a \cos(\omega t + \phi), \qquad y(t) = a \sin(\omega t + \phi)
$$

where:

- $a$ is the orbital radius  
- $T$ is the orbital period  
- $\omega$ is the angular speed (radians per second)  
- $\phi$ is the initial phase angle


In [ ]:
a = 1.0 * AU_M
T = period_s(a, sun.mass_kg)
omega = 2 * math.pi / T  # radians per second

print("For a = 1 AU:")
print("  Period (days):", T / DAY_S)
print("  Angular speed (rad/s):", omega)


---

## 7. JSON export + reload roundtrip

Export a system to JSON and reload it, verifying basic invariants.


In [ ]:
json_path = OUT_DIR / "system.json"
save_json(json_path, system)

loaded = load_json(json_path)

print("Wrote:", json_path)
print("Planet count preserved:", len(system.planets), "->", len(loaded.planets))
print("Kinds preserved:", [p.kind for p in system.planets] == [p.kind for p in loaded.planets])


**Purpose of this check:**  
Verify that a generated solar system can be exported to JSON and reloaded
without changing its structure or planet properties.

---

## 8. WaterIO checkpointing with NumPy (.npz)

Save a `(time, planet, xy)` position cube to `.npz`, reload it, and verify integrity.


In [ ]:
# Fresh system so we start from a clean phase
sun2 = Sun()
planets2 = generate_planets(sun2, 4, seed=3, inner_au=0.7, outer_au=6.0)
system2 = SolarSystem(sun=sun2, planets=planets2)

steps = 200
dt = 5 * DAY_S

pos = np.zeros((steps, len(planets2), 2), dtype=np.float64)
for t in range(steps):
    for i, p in enumerate(system2.planets):
        pos[t, i, :] = p.position_m()
    system2.step(dt)

ckpt_path = OUT_DIR / "positions_ckpt.npz"
save_checkpoint(ckpt_path, positions=pos, dt_days=np.array([dt / DAY_S]))

loaded_ckpt = load_checkpoint(ckpt_path)

print("Wrote:", ckpt_path)
print("Keys:", list(loaded_ckpt.keys()))
print("positions shape:", loaded_ckpt["positions"].shape)


**What to observe:**  
The checkpoint file should contain the saved arrays (`positions`, `dt_days`), and the
loaded positions shape should match `(time, planets, xy)`.


### 8.1 Figure: trajectories from checkpoint

We reload the checkpoint and plot trajectories again.


In [ ]:
pos_loaded = loaded_ckpt["positions"]

plt.figure()
for i, p in enumerate(planets2):
    plt.plot(pos_loaded[:, i, 0] / AU_M, pos_loaded[:, i, 1] / AU_M, label=p.name)

plt.scatter([0], [0], marker="*", s=120)
plt.xlabel("x (AU)")
plt.ylabel("y (AU)")
plt.title("Trajectories loaded from WaterIO checkpoint (.npz)")
plt.axis("equal")
plt.legend(loc="upper right", fontsize=8)
plt.show()


**What to observe:**  
The reloaded trajectories should match the expected orbit shapes, showing the checkpoint saved and loaded correctly.


### 8.2 WaterIO custom data roundtrip (write new data, reload, verify)

In addition to saving simulation outputs, WaterIO can store any NumPy arrays.
Here we create a small derived dataset from the same simulation (radius over time
for each planet), save it to `.npz`, reload it, and verify integrity.


In [ ]:
pos_loaded = loaded_ckpt["positions"]  # shape: (time, planets, xy)
radius_au = np.sqrt(pos_loaded[:, :, 0] ** 2 + pos_loaded[:, :, 1] ** 2) / AU_M  # (time, planets)

# Extra metadata we may want to keep
planet_names = np.array([p.name for p in planets2])
kinds = np.array([p.kind for p in planets2])

custom_path = OUT_DIR / "derived_data_ckpt.npz"

# Save "new" data (not just positions)
save_checkpoint(
    custom_path,
    radius_au=radius_au,
    planet_names=planet_names,
    planet_kinds=kinds,
    dt_days=loaded_ckpt["dt_days"],
)

reloaded = load_checkpoint(custom_path)

print("Wrote:", custom_path)
print("Keys:", list(reloaded.keys()))
print("radius_au shape:", reloaded["radius_au"].shape)

# Verify correctness (exact match)
print("Radius data identical:", np.allclose(reloaded["radius_au"], radius_au))
print("Names identical:", np.array_equal(reloaded["planet_names"], planet_names))
print("Kinds identical:", np.array_equal(reloaded["planet_kinds"], kinds))



**What to observe:**  
The reloaded file contains the new arrays (`radius_au`, `planet_names`, `planet_kinds`)
and the values match the original arrays, showing that WaterIO can store and restore
custom analysis results.


---

## Summary

- **SolsysGen** builds deterministic heliocentric systems using simple circular Keplerian orbits.
- Systems can be stepped forward in time, and planet positions can be inspected or visualized.
- Systems can be exported to JSON and reloaded to ensure reproducibility.
- **WaterIO** stores NumPy checkpoint arrays, such as planet positions over time, in compressed `.npz` files.

### Presentation tip

For a short presentation, run the notebook top-to-bottom once and then focus on:

- Section 4 (orbit visualization figure)
- Section 6 (Kepler equations)
- Section 8 (WaterIO checkpoint and loaded figure)
